## 로지스틱 회귀 병원 개/폐업 여부 예측

In [1]:
import pandas as pd
import numpy as np 
import scipy as sp
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import missingno as msno

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%precision 3

'%.3f'

In [2]:
# data set

train = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_병원개폐업/train.csv')
test = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_병원개폐업/test.csv')
submission = pd.read_csv('C:/Users/KIHyuk/Desktop/dacon_data/Data_병원개폐업/submission_sample.csv')

In [3]:
train.head()

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.0,3.961135e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
1,3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,515483669.0,4.472197e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.0,7.067740e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.0,4.765605e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same


In [4]:
## fisrt preprocessing

# delete missing value
train_copy = train.dropna()

In [5]:
# 파생변수 추가

train_copy['매출액_증감액'] = train_copy['revenue1'] - train_copy['revenue2']
train_copy['매출원가_증감액'] = train_copy['salescost1'] - train_copy['salescost2']
train_copy['판매비관리비_증감액'] = train_copy['sga1'] - train_copy['sga2']
train_copy['급여_증감액'] = train_copy['salary1'] - train_copy['salary2']
train_copy['영업외수익_증감액'] = train_copy['noi1'] - train_copy['noi2']
train_copy['영업외비용_증감액'] = train_copy['noe1'] - train_copy['noe2']
train_copy['이자비용_증감액'] = train_copy['interest1'] - train_copy['interest2']
train_copy['법인세비용_증감액'] = train_copy['ctax1'] - train_copy['ctax2']
train_copy['당기순이익_증감액'] = train_copy['profit1'] - train_copy['profit2']
train_copy['유동자산_증감액'] = train_copy['liquidAsset1'] - train_copy['liquidAsset2']
train_copy['당좌자산_증감액'] = train_copy['quickAsset1'] - train_copy['quickAsset2']
train_copy['미수금_증감액'] = train_copy['receivableS1'] - train_copy['receivableS2']
train_copy['재고자산_증감액'] = train_copy['inventoryAsset1'] - train_copy['inventoryAsset2']
train_copy['비유동자산_증감액'] = train_copy['nonCAsset1'] - train_copy['nonCAsset2']
train_copy['유형자산_증감액'] = train_copy['tanAsset1'] - train_copy['tanAsset2']
train_copy['기타비유동자산_증감액'] = train_copy['OnonCAsset1'] - train_copy['OnonCAsset2']
train_copy['장기미수금_증감액'] = train_copy['receivableL1'] - train_copy['receivableL2']
train_copy['부채총계_증감액'] = train_copy['debt1'] - train_copy['debt2']
train_copy['유동부채_증감액'] = train_copy['liquidLiabilities1'] - train_copy['liquidLiabilities2']
train_copy['단기차입금_증감액'] = train_copy['shortLoan1'] - train_copy['shortLoan2']
train_copy['비유동부채_증감액'] = train_copy['NCLiabilities1'] - train_copy['NCLiabilities2']
train_copy['장기차입금_증감액'] = train_copy['longLoan1'] - train_copy['longLoan2']
train_copy['순자산총계_증감액'] = train_copy['netAsset1'] - train_copy['netAsset2']
train_copy['이익잉여금_증감액'] = train_copy['surplus1'] - train_copy['surplus2']

In [6]:
# 사용하지 않는 columns delete

train_copy = train_copy.drop(['revenue1', 'salescost1', 'sga1',
       'salary1', 'noi1', 'noe1', 'interest1', 'ctax1', 'profit1',
       'liquidAsset1', 'quickAsset1', 'receivableS1', 'inventoryAsset1',
       'nonCAsset1', 'tanAsset1', 'OnonCAsset1', 'receivableL1', 'debt1',
       'liquidLiabilities1', 'shortLoan1', 'NCLiabilities1', 'longLoan1',
       'netAsset1', 'surplus1', 'revenue2', 'salescost2', 'sga2', 'salary2',
       'noi2', 'noe2', 'interest2', 'ctax2', 'profit2', 'liquidAsset2',
       'quickAsset2', 'receivableS2', 'inventoryAsset2', 'nonCAsset2',
       'tanAsset2', 'OnonCAsset2', 'receivableL2', 'debt2',
       'liquidLiabilities2', 'shortLoan2', 'NCLiabilities2', 'longLoan2',
       'netAsset2', 'surplus2'],axis=1)

# 비재무정보 제거
train_copy = train_copy.drop(['inst_id','openDate','employee1','employee2','sgg','sido','bedCount','instkind','ownerChange'],axis=1)

In [7]:
# columns check

train_copy.columns

Index(['OC', '매출액_증감액', '매출원가_증감액', '판매비관리비_증감액', '급여_증감액', '영업외수익_증감액',
       '영업외비용_증감액', '이자비용_증감액', '법인세비용_증감액', '당기순이익_증감액', '유동자산_증감액',
       '당좌자산_증감액', '미수금_증감액', '재고자산_증감액', '비유동자산_증감액', '유형자산_증감액',
       '기타비유동자산_증감액', '장기미수금_증감액', '부채총계_증감액', '유동부채_증감액', '단기차입금_증감액',
       '비유동부채_증감액', '장기차입금_증감액', '순자산총계_증감액', '이익잉여금_증감액'],
      dtype='object')

In [8]:
# 다중공선성 check

from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 10이상 VIF 제거

formula = 'OC ~ 매출액_증감액 + 매출원가_증감액 + \
       영업외수익_증감액 + 영업외비용_증감액 + 이자비용_증감액 + 법인세비용_증감액 + 당기순이익_증감액 + \
       유동자산_증감액 + 미수금_증감액 + 재고자산_증감액 + 비유동자산_증감액 + \
       기타비유동자산_증감액 + 장기미수금_증감액 + 부채총계_증감액 + 단기차입금_증감액 + \
       + 장기차입금_증감액'
y, X = dmatrices(formula, train_copy, return_type = 'dataframe')

In [9]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif.round(0).sort_values(by='VIF Factor',ascending=False)

,VIF Factor,features
1,4.0,매출액_증감액
4,4.0,영업외비용_증감액
11,3.0,비유동자산_증감액
14,3.0,부채총계_증감액
2,2.0,매출원가_증감액
3,2.0,영업외수익_증감액
7,2.0,당기순이익_증감액
8,2.0,유동자산_증감액
9,2.0,미수금_증감액
12,2.0,기타비유동자산_증감액


In [10]:
# 다중공선성 확인 후 2차 변수제거

# train_copy = train_copy.drop(['판매비관리비_증감액','급여_증감액','당좌자산_증감액','유형자산_증감액','유동부채_증감액',
#                               '비유동부채_증감액','순자산총계_증감액','이익잉여금_증감액'],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
train_copy['OC'] = label_encoder.fit_transform(train_copy['OC'])

In [21]:
train_copy

,OC,매출액_증감액,매출원가_증감액,판매비관리비_증감액,급여_증감액,영업외수익_증감액,영업외비용_증감액,이자비용_증감액,법인세비용_증감액,당기순이익_증감액,...,유형자산_증감액,기타비유동자산_증감액,장기미수금_증감액,부채총계_증감액,유동부채_증감액,단기차입금_증감액,비유동부채_증감액,장기차입금_증감액,순자산총계_증감액,이익잉여금_증감액
0,open,-8.031824e+07,0.000000e+00,-9.628707e+07,-2.995207e+07,-5.422340e+05,-14747118.0,-2446810.0,12833724.0,1.733999e+07,...,-2.557930e+07,-2449000.0,0.0,-7.616775e+07,-2.155316e+07,0.000000e+00,-5.461460e+07,-3.900000e+07,2.251697e+08,2.251697e+08
2,open,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,7.615600e+04,30000.0,0.0,0.0,4.186475e+07,...,1.204810e+08,0.0,0.0,9.241434e+07,9.241434e+07,0.000000e+00,0.000000e+00,0.000000e+00,3.005088e+08,0.000000e+00
3,open,5.649006e+09,0.000000e+00,5.753205e+09,2.074703e+09,2.941526e+07,-94104842.0,-80540715.0,-45121940.0,6.444282e+07,...,3.465022e+09,29302018.0,0.0,4.608787e+09,1.097328e+10,1.066577e+10,-6.364497e+09,-7.846960e+09,9.028303e+08,9.028303e+08
4,open,9.607370e+08,0.000000e+00,5.302526e+08,1.000740e+09,-4.853959e+08,-103018625.0,-42602315.0,0.0,4.810711e+07,...,-2.323317e+09,-567000700.0,0.0,-1.534487e+09,-1.163398e+09,3.366231e+08,-3.710896e+08,-6.080275e+08,8.074970e+07,0.000000e+00
5,open,-7.539098e+08,-6.161420e+08,-3.683077e+08,2.674107e+08,-6.026683e+07,291186440.0,-79048185.0,0.0,-1.209133e+08,...,-3.361751e+09,-54587822.0,0.0,0.000000e+00,-3.361751e+09,9.339000e+07,-2.000000e+07,0.000000e+00,0.000000e+00,-2.214085e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,open,7.796254e+09,3.198022e+09,6.153061e+09,4.192344e+09,2.923581e+09,-532533470.0,128012741.0,-414207851.0,2.647158e+09,...,3.687491e+10,0.0,0.0,1.872158e+10,3.874031e+09,0.000000e+00,1.484755e+10,1.378316e+10,7.475427e+09,0.000000e+00
293,open,3.835934e+08,4.734233e+07,2.416134e+08,2.451458e+08,2.666500e+06,-35516983.0,-65450937.0,0.0,1.328211e+08,...,7.002701e+06,12200000.0,0.0,-1.202129e+08,-1.874011e+08,0.000000e+00,6.718819e+07,9.450000e+07,3.409870e+08,0.000000e+00
294,open,1.211180e+09,-3.411295e+07,6.928005e+08,3.636481e+08,-9.955409e+06,653569466.0,12470215.0,446810960.0,-5.578436e+08,...,1.232237e+09,43475000.0,0.0,9.059307e+08,5.824931e+09,-2.813239e+08,-4.920000e+09,-5.890000e+09,4.788074e+08,0.000000e+00
295,open,-5.528116e+07,0.000000e+00,-5.223378e+07,3.499847e+07,-4.263414e+06,-5310870.0,-785309.0,-1054082.0,-9.458470e+05,...,4.837864e+06,0.0,0.0,5.333437e+08,2.463591e+08,0.000000e+00,2.869846e+08,0.000000e+00,2.456945e+08,0.000000e+00


In [24]:
# SMOTE
# 단순 반복이 아닌 KNN을 통해 주변 인공 데이터를 생성하는 SMOTE(Synthetic Minority Oversampling TEchnique)

from sklearn.neighbors import NearestNeighbors

def load_data_with_SMOTE(df):
    rawdata = df
    size = 200
    small = rawdata[rawdata['OC'] == 0] # 소수 클래스
    n_sample = small.shape[0] # 소수 클래스 개수
    idx = np.random.randint(0, n_sample, size) # 0~49범위의 랜덤수 150개 생성 
    X = small.iloc[idx, range(1,25)].values # * 랜덤수150개에 해당하는 1~5 col 정보 추출
    y = small.iloc[idx, 0].values # 랜덤수150개에 해당하는 클래스 추출 'B'
    
    # kneighbors(X) => 두가지 return (distance,index)
    knn = NearestNeighbors(n_neighbors=2) # 2개의 최근접 이웃을 찾겠다
    knn.fit(X) # 2d array input
    _d, i = knn.kneighbors(X) # input 데이터와 최근접 이웃 2가지의 index(학습데이터)와 input데이터와 이웃데이터 사이의 거리를 반환
    idx2 = i[:, 1] # train과 input이 같으므로 최근접 이웃 2가지의 return 중 첫번째는 무조건 "같은"데이터가나옴. 같은데이터 제외
    diff = X - X[idx2]
    X = X + np.random.random(24) * diff # 최근접 이웃과 비슷한 데이터로 생성
    B = np.concatenate([np.transpose(y[np.newaxis]), X], axis=1)
    B = pd.DataFrame(B)

    n_sample = rawdata[rawdata['OC'] == 1].shape[0]
    idx = np.random.randint(0, n_sample, size)
    L = rawdata[rawdata['OC'] == 1].iloc[idx]

    df = np.concatenate([B.values, L.values])

#     le = LabelEncoder()
#     X = d[:, 1:31]
#     y = le.fit_transform(d[:, 0])
    return df

In [37]:
df = load_data_with_SMOTE(train_copy)
train_res = pd.DataFrame(df,columns=train_copy.columns)

ValueError: low >= high

In [ ]:
model_1 = smf.glm(
    formula = "OC ~ 매출액_증감액 + 매출원가_증감액 + \
       영업외수익_증감액 + 영업외비용_증감액 + 이자비용_증감액 + 법인세비용_증감액 + 당기순이익_증감액 + \
       유동자산_증감액 + 미수금_증감액 + 재고자산_증감액 + 비유동자산_증감액 + \
       기타비유동자산_증감액 + 장기미수금_증감액 + 부채총계_증감액 + 단기차입금_증감액 + \
       + 장기차입금_증감액",
    data = train_res,
    family = sm.families.Binomial()
).fit()

In [ ]:
model_1.aic

In [ ]:
model_1.summary()

In [ ]:
# preprocessing function

def df_preprocessing(df):
    df = df.dropna()
    df['매출액_증감액'] = df['revenue1'] - df['revenue2']
    df['매출원가_증감액'] = df['salescost1'] - df['salescost2']
    df['판매비관리비_증감액'] = df['sga1'] - df['sga2']
    df['급여_증감액'] = df['salary1'] - df['salary2']
    df['영업외수익_증감액'] = df['noi1'] - df['noi2']
    df['영업외비용_증감액'] = df['noe1'] - df['noe2']
    df['이자비용_증감액'] = df['interest1'] - df['interest2']
    df['법인세비용_증감액'] = df['ctax1'] - df['ctax2']
    df['당기순이익_증감액'] = df['profit1'] - df['profit2']
    df['유동자산_증감액'] = df['liquidAsset1'] - df['liquidAsset2']
    df['당좌자산_증감액'] = df['quickAsset1'] - df['quickAsset2']
    df['미수금_증감액'] = df['receivableS1'] - df['receivableS2']
    df['재고자산_증감액'] = df['inventoryAsset1'] - df['inventoryAsset2']
    df['비유동자산_증감액'] = df['nonCAsset1'] - df['nonCAsset2']
    df['유형자산_증감액'] = df['tanAsset1'] - df['tanAsset2']
    df['기타비유동자산_증감액'] = df['OnonCAsset1'] - df['OnonCAsset2']
    df['장기미수금_증감액'] = df['receivableL1'] - df['receivableL2']
    df['부채총계_증감액'] = df['debt1'] - df['debt2']
    df['유동부채_증감액'] = df['liquidLiabilities1'] - df['liquidLiabilities2']
    df['단기차입금_증감액'] = df['shortLoan1'] - df['shortLoan2']
    df['비유동부채_증감액'] = df['NCLiabilities1'] - df['NCLiabilities2']
    df['장기차입금_증감액'] = df['longLoan1'] - df['longLoan2']
    df['순자산총계_증감액'] = df['netAsset1'] - df['netAsset2']
    df['이익잉여금_증감액'] = df['surplus1'] - df['surplus2']
    
    df = df.drop(['inst_id','openDate','employee1','employee2','sgg','sido','bedCount','instkind','ownerChange'],axis=1)
    df = df.drop(['revenue1', 'salescost1', 'sga1',
       'salary1', 'noi1', 'noe1', 'interest1', 'ctax1', 'profit1',
       'liquidAsset1', 'quickAsset1', 'receivableS1', 'inventoryAsset1',
       'nonCAsset1', 'tanAsset1', 'OnonCAsset1', 'receivableL1', 'debt1',
       'liquidLiabilities1', 'shortLoan1', 'NCLiabilities1', 'longLoan1',
       'netAsset1', 'surplus1', 'revenue2', 'salescost2', 'sga2', 'salary2',
       'noi2', 'noe2', 'interest2', 'ctax2', 'profit2', 'liquidAsset2',
       'quickAsset2', 'receivableS2', 'inventoryAsset2', 'nonCAsset2',
       'tanAsset2', 'OnonCAsset2', 'receivableL2', 'debt2',
       'liquidLiabilities2', 'shortLoan2', 'NCLiabilities2', 'longLoan2',
       'netAsset2', 'surplus2'],axis=1)
    
    return df